Data Pipeline Phase 1 ( Data Extraction)

In [91]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import json
import time
from urllib.parse import urljoin

In [144]:
!pip install openpyxl



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [92]:
# Constants
BASE_URL = "http://books.toscrape.com/"
CACHE_FILE = "books.json"
books = []


In [93]:
# Load cache
def load_cache():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, "r") as f:
            return json.load(f)
    return None

# Save cache
def save_cache(data):
    with open(CACHE_FILE, "w") as f:
        json.dump(data, f, indent=4)

In [94]:
# Fetch a single page and return parsed soup
def fetch_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        print("Valid, proceeding with scraping:", url)
        return bs(response.text, "html.parser")
    else:
        print("Bad request:", url)
        return None

In [95]:
def scrape_page(soup):
    for book in soup.select(".product_pod"):
        title = book.h3.a['title']
        price = book.select_one(".price_color").text.strip()
        stock = book.select_one(".availability").text.strip()
        rating = book.p['class'][1]

        books.append({
            'title': title,
            'price': price,
            'stock': stock,
            'rating': rating
        })
# Loop through all pages
def scrape_all_pages(start_url):
    url = start_url
    while url:
        soup = fetch_page(url)
        if soup is None:
            break

        scrape_page(soup)

        next_button = soup.select_one("li.next > a")
        if next_button:
            url = urljoin(url, next_button['href'])
            time.sleep(1)  # Be nice to the server
        else:
            url = None

# Main scraper function
def scraper():
    scrape_all_pages(BASE_URL)
    return books


In [96]:
# Execution starts here
cached_data = load_cache()
if cached_data:
    print("Loaded from cache")
else:
    print("Scraping website...")
    cached_data = scraper()
    if cached_data:
        save_cache(cached_data)
        print("Data saved to cache.")
    else:
        print("No valid data found.")

Loaded from cache


In [97]:
# Convert to DataFrame and print
if cached_data:
    df = pd.DataFrame(cached_data)
    print(df.head())

                                   title    price     stock rating
0                   A Light in the Attic  Â£51.77  In stock  Three
1                     Tipping the Velvet  Â£53.74  In stock    One
2                             Soumission  Â£50.10  In stock    One
3                          Sharp Objects  Â£47.82  In stock   Four
4  Sapiens: A Brief History of Humankind  Â£54.23  In stock   Five


Data Pipeline Phase 2(Transformation)

In [98]:
df

,title,price,stock,rating
0,A Light in the Attic,Â£51.77,In stock,Three
1,Tipping the Velvet,Â£53.74,In stock,One
2,Soumission,Â£50.10,In stock,One
3,Sharp Objects,Â£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,Five
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,Â£55.53,In stock,One
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Â£57.06,In stock,Four
997,A Spy's Devotion (The Regency Spies of London #1),Â£16.97,In stock,Five
998,1st to Die (Women's Murder Club #1),Â£53.98,In stock,One


In [99]:
#To strip off Duplicates
df=df.drop_duplicates()

In [100]:
df

,title,price,stock,rating
0,A Light in the Attic,Â£51.77,In stock,Three
1,Tipping the Velvet,Â£53.74,In stock,One
2,Soumission,Â£50.10,In stock,One
3,Sharp Objects,Â£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,Five
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,Â£55.53,In stock,One
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Â£57.06,In stock,Four
997,A Spy's Devotion (The Regency Spies of London #1),Â£16.97,In stock,Five
998,1st to Die (Women's Murder Club #1),Â£53.98,In stock,One


In [101]:
#To eliminate blanks
df= df.dropna()

In [102]:
# Add a Serial Number column starting from 1 and coverting it to an index
df.insert(0, 'S/N', range(1, len(df) + 1))


In [103]:
df.set_index('S/N', inplace=True)

In [104]:
df

,title,price,stock,rating
S/N,,,,
1,A Light in the Attic,Â£51.77,In stock,Three
2,Tipping the Velvet,Â£53.74,In stock,One
3,Soumission,Â£50.10,In stock,One
4,Sharp Objects,Â£47.82,In stock,Four
5,Sapiens: A Brief History of Humankind,Â£54.23,In stock,Five
...,...,...,...,...
996,Alice in Wonderland (Alice's Adventures in Won...,Â£55.53,In stock,One
997,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Â£57.06,In stock,Four
998,A Spy's Devotion (The Regency Spies of London #1),Â£16.97,In stock,Five


In [105]:
df["price"]=df["price"].str.replace("Â£","")

In [106]:
df["price"]=df["price"].astype(float)

In [107]:
#converting ratings to numerical values
df["rating"]=df["rating"].str.replace("One",'1')
df["rating"]=df["rating"].str.replace("Two",'2')
df["rating"]=df["rating"].str.replace("Three",'3')
df["rating"]=df["rating"].str.replace("Four",'4')
df["rating"]=df["rating"].str.replace("Five",'5')
df["rating"]=df["rating"].astype(int)

In [108]:
df

,title,price,stock,rating
S/N,,,,
1,A Light in the Attic,51.77,In stock,3
2,Tipping the Velvet,53.74,In stock,1
3,Soumission,50.10,In stock,1
4,Sharp Objects,47.82,In stock,4
5,Sapiens: A Brief History of Humankind,54.23,In stock,5
...,...,...,...,...
996,Alice in Wonderland (Alice's Adventures in Won...,55.53,In stock,1
997,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,In stock,4
998,A Spy's Devotion (The Regency Spies of London #1),16.97,In stock,5


Creating a Metadata

In [109]:
rating_map = {
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five"
}

df['rating_words'] = df['rating'].map(rating_map)

df['rating_words'].value_counts()


rating_words
one      226
three    203
five     196
two      196
four     179
Name: count, dtype: int64

In [110]:
df['rating'].map(rating_map)
df.groupby('rating')['price'].mean().sort_values()


rating
1    34.561195
3    34.692020
2    34.810918
5    35.374490
4    36.093296
Name: price, dtype: float64

In [141]:
# Metadata main info
main_metadata = {
    "Title": "Online Bookstore Sales Record",
    "Data Architect/Data Engineer": "Praise Gabriel",
    "Project Date": "2025-05-29",
    "Source": "Web scraped",
    "Description": "Data Engineering Project for an e-bookstore",
    "Contact": "praise@8mbtech.com"
}

main_df = pd.DataFrame(list(main_metadata.items()), columns=["Field", "Value"])

# Column descriptions
column_descriptions = {
    "S/N": "Serial number of the record",
    "Title": "Title of the book",
    "Price": "Selling price of the book (in £)"
}
columns_df = pd.DataFrame(list(column_descriptions.items()), columns=["Column", "Description"])

# Data summary
summary = pd.DataFrame({
    "Rating": df['rating'].value_counts().index,
    "Count": df['rating'].value_counts().values
})



In [142]:
main_df

,Field,Value
0,Title,Online Bookstore Sales Record
1,Data Architect/Data Engineer,Praise Gabriel
2,Project Date,2025-05-29
3,Source,Web scraped
4,Description,Data Engineering Project for an e-bookstore
5,Contact,praise@8mbtech.com


In [132]:
columns_df

,Column,Description
0,S/N,Serial number of the record
1,Title,Title of the book
2,Price,Selling price of the book (in £)


In [134]:
summary

,Rating,Count
0,1,226
1,3,203
2,5,196
3,2,196
4,4,179


Phase 3(Loading/Saving the data a CSV file)

In [145]:
with pd.ExcelWriter("bookstore_metadata.xlsx") as writer:
    main_df.to_excel(writer, sheet_name="Metadata", index=False)
    columns_df.to_excel(writer, sheet_name="Columns", index=False)
    summary.to_excel(writer, sheet_name="Rating Summary", index=False)
    df.to_excel(writer, sheet_name="Rating Summary", index=False)
